In [1]:
import numpy as np
import pandas as pd
import os
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

inline_rc = dict(mpl.rcParams)

CARGAMOS LOS DATOS DE yelp_academic_dataset_review.json

In [2]:
reviews = []
with open('data/yelp_academic_dataset_review.json') as fl:
    for i, line in enumerate(fl):
        reviews.append(json.loads(line))
        if i + 1 >= 200000:
            break

df_review = pd.DataFrame(reviews)
print(df_review.head())

              business_id  cool                 date  funny  \
0  XQfwVwDr-v0ZS3_CbbE5Xw     0  2018-07-07 22:09:11      0   
1  7ATYjTIgM3jUlt4UM3IypQ     1  2012-01-03 15:28:18      0   
2  YjUWPpI6HXG530lwP-fb2A     0  2014-02-05 20:30:30      0   
3  kxX2SOes4o-D3ZQBkiMRfA     1  2015-01-04 00:01:03      0   
4  e4Vwtrqf-wpJfwesgvdgxQ     1  2017-01-14 20:54:15      0   

                review_id  stars  \
0  KU_O5udG6zpxOg-VcAEodg    3.0   
1  BiTunyQ73aT9WBnpR9DZGw    5.0   
2  saUsX_uimxRlCVr67Z4Jig    3.0   
3  AqPFMleE6RsU23_auESxiA    5.0   
4  Sx8TMOWLNuJBWer-0pcmoA    4.0   

                                                text  useful  \
0  If you decide to eat here, just be aware it is...       0   
1  I've taken a lot of spin classes over the year...       1   
2  Family diner. Had the buffet. Eclectic assortm...       0   
3  Wow!  Yummy, different,  delicious.   Our favo...       1   
4  Cute interior and owner (?) gave us tour of up...       1   

                  u

CARGAMOS LOS DATOS DE yelp_academic_dataset_business.json

In [3]:
business = []
with open('data/yelp_academic_dataset_business.json') as fl:
    for i, line in enumerate(fl):
        business.append(json.loads(line))

df_business = pd.DataFrame(business)
print(df_business.head())

                           address  \
0           1616 Chapala St, Ste 2   
1  87 Grasso Plaza Shopping Center   
2             5255 E Broadway Blvd   
3                      935 Race St   
4                    101 Walnut St   

                                          attributes             business_id  \
0                      {'ByAppointmentOnly': 'True'}  Pns2l4eNsfO8kk83dixA6A   
1             {'BusinessAcceptsCreditCards': 'True'}  mpf3x-BjTdTEA3yCZrAYPw   
2  {'BikeParking': 'True', 'BusinessAcceptsCredit...  tUFrWirKiKi_TAnsVWINQQ   
3  {'RestaurantsDelivery': 'False', 'OutdoorSeati...  MTSW4McQd7CbVtyjqoe9mw   
4  {'BusinessAcceptsCreditCards': 'True', 'Wheelc...  mWMc6_wTdE0EUBKIGXDVfA   

                                          categories           city  \
0  Doctors, Traditional Chinese Medicine, Naturop...  Santa Barbara   
1  Shipping Centers, Local Services, Notaries, Ma...         Affton   
2  Department Stores, Shopping, Fashion, Home & G...         Tucson   
3  Res

ELIMINAMOS LAS COLUMNAS INNECESARIAS

In [4]:
df_review = df_review.drop(["review_id", "user_id", "useful", "funny", "cool", "date"], axis=1)
df_business = df_business.drop(["name", "address", "city", "state", "postal_code", "latitude", "longitude", "stars", "review_count", "is_open", "attributes", "hours"], axis=1)

COMPROBAMOS

In [5]:
print(df_review.head())
print(df_business.head())

              business_id  stars  \
0  XQfwVwDr-v0ZS3_CbbE5Xw    3.0   
1  7ATYjTIgM3jUlt4UM3IypQ    5.0   
2  YjUWPpI6HXG530lwP-fb2A    3.0   
3  kxX2SOes4o-D3ZQBkiMRfA    5.0   
4  e4Vwtrqf-wpJfwesgvdgxQ    4.0   

                                                text  
0  If you decide to eat here, just be aware it is...  
1  I've taken a lot of spin classes over the year...  
2  Family diner. Had the buffet. Eclectic assortm...  
3  Wow!  Yummy, different,  delicious.   Our favo...  
4  Cute interior and owner (?) gave us tour of up...  
              business_id                                         categories
0  Pns2l4eNsfO8kk83dixA6A  Doctors, Traditional Chinese Medicine, Naturop...
1  mpf3x-BjTdTEA3yCZrAYPw  Shipping Centers, Local Services, Notaries, Ma...
2  tUFrWirKiKi_TAnsVWINQQ  Department Stores, Shopping, Fashion, Home & G...
3  MTSW4McQd7CbVtyjqoe9mw  Restaurants, Food, Bubble Tea, Coffee & Tea, B...
4  mWMc6_wTdE0EUBKIGXDVfA                          Brewpubs, Breweri